## Import

In [ ]:
import importlib

import torch
from torch_geometric import data, loader
import numpy as np

from awe import utils, filtering, features, html_utils, awe_graph, visual
from awe.data import swde, live, dataset

for module in [utils, filtering, dataset, swde, live, features, html_utils, awe_graph, visual]:
    importlib.reload(module)

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
sds = swde.Dataset(suffix='-exact')
#sds.validate()
ds = dataset.DatasetCollection()

## Split data

In [ ]:
vertical = sds.verticals[0]
dev_pages = vertical.websites[0].pages[:400] + vertical.websites[1].pages[:100]
train_pages, val_pages = utils.train_val_split(dev_pages, .2)
unseen_pages = vertical.websites[2].pages[:100]
ds.create('train', train_pages)
ds.create('val', val_pages)
ds.create('unseen', unseen_pages)
ds.get_lengths()

## Extract features

In [ ]:
ds.features = [
    features.Depth(),
    features.IsLeaf(),
    features.CharCategories(),
    features.FontSize(),
    features.WordEmbedding(),
]

In [ ]:
ds.feature_dim, ds.feature_labels

In [ ]:
# Uncomment next line to invalidate computed features.
#ds.delete_saved_features()

In [ ]:
ds.initialize()

In [ ]:
ds.parallelize = None
ds['train'].prepare()
ds['val'].prepare()
ds['unseen'].prepare()

In [ ]:
ds.first_dataset.label_map

In [ ]:
ds.count_labels()

## Create dataloaders

In [ ]:
BATCH_SIZE = 1
ds['train'].loader = loader.DataLoader(ds['train'], batch_size=BATCH_SIZE, shuffle=True)
ds['val'].loader = loader.DataLoader(ds['val'], batch_size=BATCH_SIZE)
ds['unseen'].loader = loader.DataLoader(ds['unseen'], batch_size=BATCH_SIZE)

In [ ]:
for batch in ds['train'].loader:
    print(batch)
    break

In [ ]:
len(ds['train'].loader) + len(ds['val'].loader)

## Inspect data

In [ ]:
interesting_nodes = (
    (node, x, y)
    for node, x, y in ds['val'].iterate_data()
    if node.labels == ['price']
)
next(iter(interesting_nodes))

In [ ]:
# Are all labeled nodes also leaf nodes?
from tqdm.auto import tqdm
def verify_leaf_nodes(names):
    for name in names:
        for page in tqdm(ds[name].pages, desc=name):
            ctx = ds.create_context(page)
            for node in ctx.nodes:
                if len(node.labels) != 0 and not node.is_text:
                    print(f'Node {node.xpath} with labels {node.labels} in page {page.identifier} is not leaf.')
                    return
#verify_leaf_nodes(['train', 'val', 'unseen'])

## Weight labels

In [ ]:
def count_label(data: list[data.Data], label: int):
    return sum(1 for d in data for y in d.y if y == label)

def count_labels(data: list[data.Data]):
    return [count_label(data, label) for label in ds.first_dataset.label_map.values()]

In [ ]:
label_counts = count_labels(ds['train'])
{
    label: count
    for label, count in
    zip(ds.first_dataset.label_map.keys(), label_counts)
}

In [ ]:
#label_weights = [len(ds['train']) / count for count in label_counts]
#label_weights

In [ ]:
# Manual override
label_count = len(ds.first_dataset.label_map)
label_weights = [1] + [100_000] * (label_count - 1)
label_weights

## Train a model

In [ ]:
import pytorch_lightning as pl

from awe import awe_model, gym, utils
for module in [awe_model, gym, utils]:
    importlib.reload(module)

In [ ]:
model = awe_model.AweModel(ds.feature_dim, label_count, label_weights)
model

In [ ]:
g = gym.Gym(ds, model)
# Comment next line to restore previously trained model.
g.restore_checkpoint = False
g.get_last_checkpoint_path(), g.get_last_checkpoint_version()

In [ ]:
g.trainer = pl.Trainer(
    max_epochs=25,
    callbacks=[gym.CustomProgressBar(refresh_rate=100)],
    resume_from_checkpoint=g.get_last_checkpoint_path(),
    logger=g.create_logger(),
)

In [ ]:
g.trainer.logger.version

In [ ]:
g.trainer.fit(model, ds['train'].loader, ds['val'].loader)

In [ ]:
# TODO: Breaks torch_geometric for some reason!
#g.save_model()

In [ ]:
g.save_model_text()

In [ ]:
g.save_results('val')

In [ ]:
g.save_results('unseen')

In [ ]:
g.save_pages()

In [ ]:
# Uncomment and change name to save interesting results.
#g.save_named_version('gnn-d-64')

## Example prediction

In [ ]:
from awe import predictor
importlib.reload(predictor)

In [ ]:
val_predictor = predictor.Predictor(ds, 'unseen', model)

In [ ]:
predict_total = len(val_predictor.items)
predict_indices = [0, 1, predict_total - 2, predict_total - 1]

In [ ]:
val_predictor.evaluate(predict_indices)

In [ ]:
val_predictor.get_example_texts(predict_indices)

## Live prediction

In [ ]:
urls = [
    'https://www.cars.com/vehicledetail/81d8ee1f-155e-44ec-8ea4-0b25b0ca608a/'
]
live_pages = [live.Page(url) for url in urls]

In [ ]:
# Download pages.
[page.dom for page in live_pages]

In [ ]:
ds.create('live', live_pages).prepare(skip_existing=False)

In [ ]:
live_predictor = predictor.Predictor(ds, 'live', model)

In [ ]:
live_predictor.get_example_texts(range(len(live_pages)))